<a href="https://colab.research.google.com/github/LucasAlegre/vote-network/blob/master/vote_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install and import libraries (need to restart the session after install)

In [1]:
! pip install --user graphistry
! pip install python-igraph
! pip install netbone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.8/439.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.3/332.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 50.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 22.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 986.3/986.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.2/77.2 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18

In [1]:
import graphistry
import pandas as pd
import urllib.request
import requests
from datetime import date
import numpy as np
import igraph as ig
from igraph import Graph, summary
import networkx as nx
import netbone
from netbone import disparity
from netbone.filters import threshold_filter, fraction_filter
from itertools import combinations
import os
# Init graphistry
graphistry.register(api=3, protocol="https", server="hub.graphistry.com", username="LucasAlegre", password="")

DEBUG:jax._src.path:etils.epath found. Using etils.epath for file I/O.
DEBUG:graphistry.pygraphistry:@PyGraphistry login : org_name :None vs PyGraphistry.org_name() : None
DEBUG:graphistry.arrow_uploader:2. @ArrowUploader.__init__: After set self.org_name: None, self.__org_name : None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): hub.graphistry.com:443
DEBUG:urllib3.connectionpool:https://hub.graphistry.com:443 "POST /api-token-auth/ HTTP/1.1" 201 734
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): hub.graphistry.com:443
DEBUG:urllib3.connectionpool:https://hub.graphistry.com:443 "POST /api/v2/o/lucasalegre/switch/ HTTP/1.1" 200 180
DEBUG:graphistry.pygraphistry:@PyGraphistry login After ArrowUploader.login: org_name :None vs PyGraphistry.org_name() : lucasalegre
DEBUG:graphistry.pygraphistry:1. @PyGraphistry refresh, org_name: lucasalegre
DEBUG:graphistry.pygraphistry:JWT refresh via token
DEBUG:graphistry.arrow_uploader:2. @ArrowUploader.__init__: Af

## Data Retrieval and Processing

In [4]:
start_date = '2002-01-01' #@param {type:"date"}
end_date = '2006-12-31' #@param {type:"date"}
start_date = pd.to_datetime(start_date)
end_date = pd.to_datetime(end_date)

In [77]:
# Collect all votes between start and end dates
os.makedirs('data', exist_ok=True)
for year in range(start_date.year, end_date.year + 1):
    # Read or download votes
    path = 'data/votacoesVotos-{}.csv'.format(year)
    if not os.path.isfile(path):
      headers = {'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'}
      url = 'https://dadosabertos.camara.leg.br/arquivos/votacoesVotos/csv/votacoesVotos-{}.csv'.format(year)
      request = urllib.request.Request(url, None, headers) #The assembled request
      response = urllib.request.urlopen(request)
      data = response.read() # The data u need
      with open(path, 'wb') as f:
          f.write(data)

    path_themes = 'data/proposicoesTemas-{}.csv'.format(year)
    if not os.path.isfile(path_themes):
      headers = {'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'}
      url = 'https://dadosabertos.camara.leg.br/arquivos/proposicoesTemas/csv/proposicoesTemas-{}.csv'.format(year)
      request = urllib.request.Request(url, None, headers) #The assembled request
      response = urllib.request.urlopen(request)
      data = response.read() # The data u need
      with open(path_themes, 'wb') as f:
          f.write(data)

    path_propositions = 'data/proposicoes-{}.csv'.format(year)
    if not os.path.isfile(path_propositions):
      headers = {'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'}
      url = 'https://dadosabertos.camara.leg.br/arquivos/proposicoes/csv/proposicoes-{}.csv'.format(year)
      request = urllib.request.Request(url, None, headers) #The assembled request
      response = urllib.request.urlopen(request)
      data = response.read() # The data u need
      with open(path_propositions, 'wb') as f:
          f.write(data)

    path_votings = 'data/votacoes-{}.csv'.format(year)
    if not os.path.isfile(path_votings):
      headers = {'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'}
      url = 'https://dadosabertos.camara.leg.br/arquivos/votacoes/csv/votacoes-{}.csv'.format(year)
      request = urllib.request.Request(url, None, headers) #The assembled request
      response = urllib.request.urlopen(request)
      data = response.read() # The data u need
      with open(path_votings, 'wb') as f:
          f.write(data)

    vote_data = pd.read_csv(path, sep=';')
    theme_data = pd.read_csv(path_themes, sep=';')
    proposition_data = pd.read_csv(path_propositions, sep=';')
    votings_data = pd.read_csv(path_votings, sep=';')

    # Remove simbolic votes
    vote_data = vote_data[vote_data['voto'] != 'Simbólico']

    # Filter dates
    vote_data['dataHoraVoto'] = pd.to_datetime(vote_data['dataHoraVoto'])
    vote_data = vote_data.loc[(vote_data['dataHoraVoto'] >= start_date) & (vote_data['dataHoraVoto'] <= end_date)]
    vote_data.sort_values(by=['dataHoraVoto'], inplace=True)

    if year == start_date.year:
        votes = vote_data
        themes = theme_data
        propositions = proposition_data
        votings = votings_data
    else:
        votes = pd.concat([votes, vote_data])
        themes = pd.concat([themes, theme_data])
        propositions = pd.concat([propositions, proposition_data])
        votings = pd.concat([votings, votings_data])
votes

,idVotacao,uriVotacao,dataHoraVoto,voto,deputado_id,deputado_uri,deputado_nome,deputado_siglaPartido,deputado_uriPartido,deputado_siglaUf,deputado_idLegislatura,deputado_urlFoto
3,14541-69,https://dadosabertos.camara.leg.br/api/v2/vota...,2002-02-26 20:09:10,Artigo 17,74646.0,https://dadosabertos.camara.leg.br/api/v2/depu...,Aécio Neves,PSDB,https://dadosabertos.camara.leg.br/api/v2/part...,MG,51,https://www.camara.leg.br/internet/deputado/ba...
211,14541-69,https://dadosabertos.camara.leg.br/api/v2/vota...,2002-02-26 20:38:39,Sim,74584.0,https://dadosabertos.camara.leg.br/api/v2/depu...,José Militão,PSDB,https://dadosabertos.camara.leg.br/api/v2/part...,MG,51,https://www.camara.leg.br/internet/deputado/ba...
263,14541-69,https://dadosabertos.camara.leg.br/api/v2/vota...,2002-02-26 20:38:40,Sim,74744.0,https://dadosabertos.camara.leg.br/api/v2/depu...,Márcio Reinaldo Moreira,PPB,https://dadosabertos.camara.leg.br/api/v2/part...,MG,51,https://www.camara.leg.br/internet/deputado/ba...
25,14541-69,https://dadosabertos.camara.leg.br/api/v2/vota...,2002-02-26 20:38:40,Sim,74699.0,https://dadosabertos.camara.leg.br/api/v2/depu...,Angela Guadagnin,PT,https://dadosabertos.camara.leg.br/api/v2/part...,SP,51,https://www.camara.leg.br/internet/deputado/ba...
270,14541-69,https://dadosabertos.camara.leg.br/api/v2/vota...,2002-02-26 20:38:41,Sim,74177.0,https://dadosabertos.camara.leg.br/api/v2/depu...,Marinha Raupp,PSDB,https://dadosabertos.camara.leg.br/api/v2/part...,RO,51,https://www.camara.leg.br/internet/deputado/ba...
...,...,...,...,...,...,...,...,...,...,...,...,...
19452,335039-55,https://dadosabertos.camara.leg.br/api/v2/vota...,2006-12-20 18:47:58,Não,74848.0,https://dadosabertos.camara.leg.br/api/v2/depu...,Jandira Feghali,PCdoB,https://dadosabertos.camara.leg.br/api/v2/part...,RJ,52,https://www.camara.leg.br/internet/deputado/ba...
19659,335039-55,https://dadosabertos.camara.leg.br/api/v2/vota...,2006-12-20 18:48:06,Sim,74392.0,https://dadosabertos.camara.leg.br/api/v2/depu...,Zonta,PP,https://dadosabertos.camara.leg.br/api/v2/part...,SC,52,https://www.camara.leg.br/internet/deputado/ba...
19555,335039-55,https://dadosabertos.camara.leg.br/api/v2/vota...,2006-12-20 18:48:16,Sim,74318.0,https://dadosabertos.camara.leg.br/api/v2/depu...,Mussa Demes,PFL,https://dadosabertos.camara.leg.br/api/v2/part...,PI,52,https://www.camara.leg.br/internet/deputado/ba...
19371,335039-55,https://dadosabertos.camara.leg.br/api/v2/vota...,2006-12-20 18:49:00,Sim,74181.0,https://dadosabertos.camara.leg.br/api/v2/depu...,Chicão Brígido,PMDB,https://dadosabertos.camara.leg.br/api/v2/part...,AC,52,https://www.camara.leg.br/internet/deputado/ba...


In [78]:
# Adiciona temas as proposições
propositions = propositions.merge(themes, left_on='uri', right_on='uriProposicao', how='left')
propositions

,id,uri,siglaTipo_x,numero_x,ano_x,codTipo,descricaoTipo,ementa,ementaDetalhada,keywords,...,ultimoStatus_despacho,ultimoStatus_apreciacao,ultimoStatus_url,uriProposicao,siglaTipo_y,numero_y,ano_y,codTema,tema,relevancia
0,29725,https://dadosabertos.camara.leg.br/api/v2/prop...,EMR,1,0,257,Emenda de Relator,NaN,NaN,NaN,...,Arquivada nos termos do Artigo 105 do Regiment...,Indefinida,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,42731,https://dadosabertos.camara.leg.br/api/v2/prop...,EMR,1,0,257,Emenda de Relator,NaN,NaN,NaN,...,Arquivada nos termos do Artigo 105 do Regiment...,Indefinida,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,42896,https://dadosabertos.camara.leg.br/api/v2/prop...,PAR,1,0,187,Parecer de Comissão,NaN,NaN,NaN,...,Arquivado nos termos do Artigo 105 do Regiment...,Indefinida,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,42917,https://dadosabertos.camara.leg.br/api/v2/prop...,PAR,1,0,187,Parecer de Comissão,NaN,NaN,NaN,...,Arquivado nos termos do Artigo 105 do Regiment...,Indefinida,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,43291,https://dadosabertos.camara.leg.br/api/v2/prop...,PAR,1,0,187,Parecer de Comissão,NaN,NaN,NaN,...,Arquivado nos termos do Artigo 105 do Regiment...,Indefinida,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125125,1280645,https://dadosabertos.camara.leg.br/api/v2/prop...,SBT-A,1,0,701,Substitutivo adotado pela Comissão,NaN,NaN,NaN,...,Apresentação do Substitutivo adotado pela Comi...,Indefinida,https://www.camara.leg.br/proposicoesWeb/prop_...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125126,2129921,https://dadosabertos.camara.leg.br/api/v2/prop...,EMS,7310,2006,132,Emenda/Substitutivo do Senado,Substitutivo do Senado Federal ao Projeto de L...,NaN,"Criação, Dia Nacional da Ikebana, comemoração,...",...,Apresentação da Emenda/Substitutivo do Senado ...,Proposição Sujeita à Apreciação do Plenário,https://www.camara.leg.br/proposicoesWeb/prop_...,https://dadosabertos.camara.leg.br/api/v2/prop...,EMS,7310.0,2006.0,35.0,"Arte, Cultura e Religião",0.0
125127,2129921,https://dadosabertos.camara.leg.br/api/v2/prop...,EMS,7310,2006,132,Emenda/Substitutivo do Senado,Substitutivo do Senado Federal ao Projeto de L...,NaN,"Criação, Dia Nacional da Ikebana, comemoração,...",...,Apresentação da Emenda/Substitutivo do Senado ...,Proposição Sujeita à Apreciação do Plenário,https://www.camara.leg.br/proposicoesWeb/prop_...,https://dadosabertos.camara.leg.br/api/v2/prop...,EMS,7310.0,2006.0,72.0,Homenagens e Datas Comemorativas,0.0
125128,2167560,https://dadosabertos.camara.leg.br/api/v2/prop...,SBT-A,1,0,701,Substitutivo adotado pela Comissão,NaN,NaN,NaN,...,Apresentação do Substitutivo adotado pela Comi...,Indefinida,https://www.camara.leg.br/proposicoesWeb/prop_...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
votings = votings[votings["ultimaApresentacaoProposicao_uriProposicao"].notna()]
votings

,id,uri,data,dataHoraRegistro,idOrgao,uriOrgao,siglaOrgao,idEvento,uriEvento,aprovacao,votosSim,votosNao,votosOutros,descricao,ultimaAberturaVotacao_dataHoraRegistro,ultimaAberturaVotacao_descricao,ultimaApresentacaoProposicao_dataHoraRegistro,ultimaApresentacaoProposicao_descricao,ultimaApresentacaoProposicao_idProposicao,ultimaApresentacaoProposicao_uriProposicao
1,37007-23,https://dadosabertos.camara.leg.br/api/v2/vota...,2002-02-19,NaN,180,https://dadosabertos.camara.leg.br/api/v2/orga...,PLEN,1815,https://dadosabertos.camara.leg.br/api/v2/even...,1,0,0,0,Aprovada a Redação Final oferecida pelo Relato...,NaN,Votação da Redação Final.,2006-08-23T11:49:49,"Parecer proferido em Plenário pelo Relator, De...",68258,https://dadosabertos.camara.leg.br/api/v2/prop...
2,37007-20,https://dadosabertos.camara.leg.br/api/v2/vota...,2002-02-19,NaN,180,https://dadosabertos.camara.leg.br/api/v2/orga...,PLEN,1815,https://dadosabertos.camara.leg.br/api/v2/even...,0,0,0,0,Rejeitada a Emenda da Comissão Mista do Congre...,NaN,Votação em turno único.,2006-08-23T11:49:49,"Parecer proferido em Plenário pelo Relator, De...",68258,https://dadosabertos.camara.leg.br/api/v2/prop...
3,37007-21,https://dadosabertos.camara.leg.br/api/v2/vota...,2002-02-19,NaN,180,https://dadosabertos.camara.leg.br/api/v2/orga...,PLEN,1815,https://dadosabertos.camara.leg.br/api/v2/even...,1,0,0,0,Aprovada a Medida Provisória.,NaN,Votação em turno único.,2006-08-23T11:49:49,"Parecer proferido em Plenário pelo Relator, De...",68258,https://dadosabertos.camara.leg.br/api/v2/prop...
6,37810-15,https://dadosabertos.camara.leg.br/api/v2/vota...,2002-02-20,NaN,180,https://dadosabertos.camara.leg.br/api/v2/orga...,PLEN,1818,https://dadosabertos.camara.leg.br/api/v2/even...,1,0,0,0,Aprovado o parecer do relator da CM pela inadm...,NaN,Votação em turno único.,2006-09-26T10:11:57,"Parecer proferido em Plenário pelo Relator, De...",68276,https://dadosabertos.camara.leg.br/api/v2/prop...
15,32428-43,https://dadosabertos.camara.leg.br/api/v2/vota...,2002-02-20,NaN,180,https://dadosabertos.camara.leg.br/api/v2/orga...,PLEN,1820,https://dadosabertos.camara.leg.br/api/v2/even...,1,0,0,0,"Aprovado o Substitutivo, ressalvados os destaq...",NaN,Votação em primeiro turno.,2005-05-31T20:44:13,"Apresentação do Recurso, REC 206/2002, pelo De...",44312,https://dadosabertos.camara.leg.br/api/v2/prop...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4771,285354-21,https://dadosabertos.camara.leg.br/api/v2/vota...,2006-12-21,2006-12-21T12:36:08,2003,https://dadosabertos.camara.leg.br/api/v2/orga...,CCJC,14050,https://dadosabertos.camara.leg.br/api/v2/even...,1,0,0,0,Aprovado o Parecer contra o voto do Deputado D...,NaN,NaN,2006-12-08T10:09:10,"Parecer do Relator, Dep. Sérgio Miranda (PDT-M...",337989,https://dadosabertos.camara.leg.br/api/v2/prop...
4772,19936-79,https://dadosabertos.camara.leg.br/api/v2/vota...,2006-12-21,2006-12-21T12:50:15,2003,https://dadosabertos.camara.leg.br/api/v2/orga...,CCJC,14050,https://dadosabertos.camara.leg.br/api/v2/even...,1,0,0,0,Aprovado por Unanimidade o Parecer,NaN,NaN,2006-11-22T19:32:56,"Parecer do Relator, Dep. Léo Alcântara (PSDB-C...",336621,https://dadosabertos.camara.leg.br/api/v2/prop...
4773,156400-58,https://dadosabertos.camara.leg.br/api/v2/vota...,2006-12-21,2006-12-21T13:14:24,2003,https://dadosabertos.camara.leg.br/api/v2/orga...,CCJC,14050,https://dadosabertos.camara.leg.br/api/v2/even...,1,0,0,0,Aprovado por Unanimidade o Parecer,NaN,NaN,2006-11-30T17:40:46,"Parecer do Relator, Dep. Antônio Carlos Biffi ...",337316,https://dadosabertos.camara.leg.br/api/v2/prop...
4774,308546-22,https://dadosabertos.camara.leg.br/api/v2/vota...,2006-12-21,2006-12-21T13:14:53,2003,https://dadosabertos.camara.leg.br/api/v2/orga...,CCJC,14050,https://dadosabertos.camara.leg.br/api/v2/even...,1,0,0,0,Aprovado por Unanimidade o Parecer,NaN,NaN,2006-06-28T17:32:15,"Parecer do Relator, Dep. Paulo Afonso (PMDB-SC...",328800,https://

In [81]:
# Liga cada votação com sua proposição
votings = votings.merge(propositions, left_on='ultimaApresentacaoProposicao_uriProposicao', right_on="uri", how='left')

In [82]:
votes = votes.merge(votings, left_on="uriVotacao", right_on="uri_x", how="left")
votes

,idVotacao,uriVotacao,dataHoraVoto,voto,deputado_id,deputado_uri,deputado_nome,deputado_siglaPartido,deputado_uriPartido,deputado_siglaUf,...,ultimoStatus_despacho,ultimoStatus_apreciacao,ultimoStatus_url,uriProposicao,siglaTipo_y,numero_y,ano_y,codTema,tema,relevancia
0,14541-69,https://dadosabertos.camara.leg.br/api/v2/vota...,2002-02-26 20:09:10,Artigo 17,74646.0,https://dadosabertos.camara.leg.br/api/v2/depu...,Aécio Neves,PSDB,https://dadosabertos.camara.leg.br/api/v2/part...,MG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,14541-69,https://dadosabertos.camara.leg.br/api/v2/vota...,2002-02-26 20:38:39,Sim,74584.0,https://dadosabertos.camara.leg.br/api/v2/depu...,José Militão,PSDB,https://dadosabertos.camara.leg.br/api/v2/part...,MG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,14541-69,https://dadosabertos.camara.leg.br/api/v2/vota...,2002-02-26 20:38:40,Sim,74744.0,https://dadosabertos.camara.leg.br/api/v2/depu...,Márcio Reinaldo Moreira,PPB,https://dadosabertos.camara.leg.br/api/v2/part...,MG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,14541-69,https://dadosabertos.camara.leg.br/api/v2/vota...,2002-02-26 20:38:40,Sim,74699.0,https://dadosabertos.camara.leg.br/api/v2/depu...,Angela Guadagnin,PT,https://dadosabertos.camara.leg.br/api/v2/part...,SP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,14541-69,https://dadosabertos.camara.leg.br/api/v2/vota...,2002-02-26 20:38:41,Sim,74177.0,https://dadosabertos.camara.leg.br/api/v2/depu...,Marinha Raupp,PSDB,https://dadosabertos.camara.leg.br/api/v2/part...,RO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78073,335039-55,https://dadosabertos.camara.leg.br/api/v2/vota...,2006-12-20 18:47:58,Não,74848.0,https://dadosabertos.camara.leg.br/api/v2/depu...,Jandira Feghali,PCdoB,https://dadosabertos.camara.leg.br/api/v2/part...,RJ,...,"Parecer proferido em Plenário pelo Relator, De...",Indefinida,https://www.camara.leg.br/proposicoesWeb/prop_...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78074,335039-55,https://dadosabertos.camara.leg.br/api/v2/vota...,2006-12-20 18:48:06,Sim,74392.0,https://dadosabertos.camara.leg.br/api/v2/depu...,Zonta,PP,https://dadosabertos.camara.leg.br/api/v2/part...,SC,...,"Parecer proferido em Plenário pelo Relator, De...",Indefinida,https://www.camara.leg.br/proposicoesWeb/prop_...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78075,335039-55,https://dadosabertos.camara.leg.br/api/v2/vota...,2006-12-20 18:48:16,Sim,74318.0,https://dadosabertos.camara.leg.br/api/v2/depu...,Mussa Demes,PFL,https://dadosabertos.camara.leg.br/api/v2/part...,PI,...,"Parecer proferido em Plenário pelo Relator, De...",Indefinida,https://www.camara.leg.br/proposicoesWeb/prop_...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78076,335039-55,https://dadosabertos.camara.leg.br/api/v2/vota...,2006-12-20 18:49:00,Sim,74181.0,https://dadosabertos.camara.leg.br/api/v2/depu...,Chicão Brígido,PMDB,https://dadosabertos.camara.leg.br/api/v2/part...,AC,...,"Parecer proferido em Plenário pelo Relator, De...",Indefinida,https://www.camara.leg.br/proposicoesWeb/prop_...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def get_total_expenses(deputy_id, start_date, end_date):
    #expenses = []
    total = 0.0
    years = '&'.join(['ano={}'.format(year) for year in range(start_date.year, end_date.year+1)])
    url = "https://dadosabertos.camara.leg.br/api/v2/deputados/{}/despesas?{}&itens=100000&ordem=ASC&ordenarPor=ano".format(deputy_id, years)
    read_all = False
    while not read_all:
        read_all = True
        page = requests.get(url).json()
        if 'dados' not in page:
            break
        total += sum([e['valorDocumento'] for e in page['dados']])
        #this_expenses = [{'tipo': e['tipoDespesa'], 'valor': e['valorDocumento']} for e in page['dados']]
        #expenses.extend(this_expenses)
        for link in page['links']:
            if link['rel'] == 'next':
                url = link['href']
                read_all = False
                break
    return total

def get_deputy_info(deputy_id):
    url = 'https://dadosabertos.camara.leg.br/api/v2/deputados/{}'.format(deputy_id)
    data = requests.get(url).json()['dados']
    ultimoStatus = data['ultimoStatus']
    today = date.today()
    nascimento = pd.to_datetime(data['dataNascimento'])
    idade = today.year - nascimento.year - ((today.month, today.day) < (nascimento.month, nascimento.day))
    return {'sexo': data['sexo'],
            'escolaridade': data['escolaridade'],
            'idade': idade,
            'e-mail': ultimoStatus['email'],
            'situação': ultimoStatus['situacao'],
            'cidade natal': data['municipioNascimento']}

In [ ]:
#%% Take care of different names for same deputy
for group, df_group in votes.groupby('deputado_id'):
    votes['deputado_nome'].loc[votes['deputado_id'] == group] = sorted(df_group['deputado_nome'].unique())[0]

#%% Partidos que mudaram de nome
votes['deputado_siglaPartido'].replace('PMDB', 'MDB', inplace=True)
votes['deputado_siglaPartido'].replace('PRB', 'REPUBLICANOS', inplace=True)
votes['deputado_siglaPartido'].replace('PR', 'PL', inplace=True)
votes['deputado_siglaPartido'].replace('PATRIOTA', 'PATRI', inplace=True)
votes['deputado_siglaPartido'].replace('PPS', 'CIDADANIA', inplace=True)

# Logo dos partidos
party_logo = {}
for p in votes['deputado_uriPartido'].unique():
    if not pd.isna(p):
        dados = requests.get(p).json()['dados']
        if dados['sigla'] == 'MDB':
            party_logo[p] = 'https://logodownload.org/wp-content/uploads/2018/04/mdb-logo-partido.png'
        elif dados['sigla'] == 'SOLIDARIEDADE':
            party_logo[p] = 'https://upload.wikimedia.org/wikipedia/commons/f/fe/Logomarca_do_Partido_Solidariedade.png'
        elif dados['sigla'] == 'PATRI':
            party_logo[p] = 'https://upload.wikimedia.org/wikipedia/en/8/8e/Patriota_logo.png'
        elif dados['sigla'] == 'REPUBLICANOS':
            party_logo[p] = 'https://upload.wikimedia.org/wikipedia/en/0/0d/Republicanos_logo.png'
        elif dados['sigla'] == 'PL':
            party_logo[p] = 'https://upload.wikimedia.org/wikipedia/commons/0/03/PL-logo.jpg'
        elif dados['sigla'] == 'CIDADANIA':
            party_logo[p] = 'https://upload.wikimedia.org/wikipedia/commons/d/d7/Logo_do_Cidadania_23.png'
        elif dados['sigla'] == 'NOVO':
            party_logo[p] = 'https://upload.wikimedia.org/wikipedia/commons/b/b5/Novo30_AOC.png'
        elif dados['sigla'] == 'UNIÃO':
            party_logo[p] = 'https://upload.wikimedia.org/wikipedia/commons/7/73/Uni%C3%A3o_Brasil_logo.svg'
        else:
            party_logo[p] = dados['urlLogo']

#all_data['deputado_siglaPartido'].replace('PPL', np.nan, inplace=True) # PPL foi incorporado
#all_data['deputado_siglaPartido'].replace('PRP', np.nan, inplace=True) # PRP foi incorporado
#all_data['deputado_siglaPartido'].replace('PHS', np.nan, inplace=True) # PHS foi incorporado
# all_data = pd.merge(all_data, motions_themes, on="idVotacao", how="inner")
#all_data['deputado_siglaPartido'].fillna('S.PART.', inplace=True)
# all_data.groupby('idVotacao')['voto'].count()

#all_data.to_csv('votos_{}_to_{}.csv'.format(start_date, end_date), index=False)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): dadosabertos.camara.leg.br:443
DEBUG:urllib3.connectionpool:https://dadosabertos.camara.leg.br:443 "GET /api/v2/partidos/37903 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): dadosabertos.camara.leg.br:443
DEBUG:urllib3.connectionpool:https://dadosabertos.camara.leg.br:443 "GET /api/v2/partidos/36898 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): dadosabertos.camara.leg.br:443
DEBUG:urllib3.connectionpool:https://dadosabertos.camara.leg.br:443 "GET /api/v2/partidos/36839 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): dadosabertos.camara.leg.br:443
DEBUG:urllib3.connectionpool:https://dadosabertos.camara.leg.br:443 "GET /api/v2/partidos/37908 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): dadosabertos.camara.leg.br:443
DEBUG:urllib3.connectionpool:https://dadosabertos.camara.leg.br:4

## Graph Construction

In [ ]:
def generalized_similarity(m, min_eps=0.001, max_iter=1000):
    """ Balázs Kovács, "A generalized model of relational similarity," Social Networks, 32(3), July 2010, pp. 197–211
        Based on: https://github.com/dzinoviev/generalizedsimilarity
    """
    arcs0 = m - m.mean(axis=1)[:, np.newaxis]
    arcs1 = m.T - m.mean(axis=0)[:, np.newaxis]

    eps = min_eps + 1
    N = np.eye(m.shape[1])

    iters = 0
    while (eps > min_eps and iters < max_iter) or np.isnan(N).any():
        M = arcs0.dot(N).dot(arcs0.T)
        m = np.sqrt(M.diagonal())
        M = ((M / (m+1e-8)).T / (m+1e-8)).T

        Np = arcs1.dot(M).dot(arcs1.T)
        n = np.sqrt(Np.diagonal())
        Np = ((Np / (n+1e-8)).T / (n+1e-8)).T
        eps = np.abs(Np - N).max()
        N = Np

        iters += 1
    return M

def pearson_correlation(m):
    arcs0 = m - m.mean(axis=1)[:, np.newaxis]
    arcs1 = m.T - m.mean(axis=0)[:, np.newaxis]
    M = arcs0.dot(arcs0.T)
    m = np.sqrt(M.diagonal())
    M = ((M / m).T / m).T
    return M

def filter_edges(edges_list, num_nodes, threshold=None, density=0.1):
    edges, weights = [], []
    if threshold is not None:
        for e in edges_list:
            if e[1] >= threshold:
                edges.append(e[0])
                weights.append(e[1])
    else:
        count = int(num_nodes * (num_nodes - 1) * density / 2)
        edges_list.sort(reverse=True, key=lambda e: e[1])
        edges_list = edges_list[:count]
        edges = [e[0] for e in edges_list]
        weights = [e[1] for e in edges_list]
    return edges, weights

In [ ]:
# Vote Matrix
date_intervals = [('2018-01-01', '2018-12-31'), ('2019-01-01', '2019-12-31'), ('2020-01-01', '2020-12-31'), ('2021-01-01', '2021-12-31'),
                  ('2022-01-01', '2022-12-31'), ('2023-01-01', '2023-12-31'), ('2024-01-01', '2024-12-31'), ('2025-01-01', '2025-12-31')]

reps = votes['deputado_nome'].unique()
num_reps = len(reps)
rep_to_ind = {reps[i]: i for i in range(len(reps))}
parties = [p for p in votes['deputado_siglaPartido'].unique() if pd.notna(p)]

def get_vote_matrix(votes, start_date, end_date):
  vote_data = votes.loc[(votes['dataHoraVoto'] >= start_date) & (votes['dataHoraVoto'] <= end_date)]
  vote_data.sort_values(by=['dataHoraVoto'], inplace=True)
  motions = vote_data['idVotacao'].unique()
  motion_to_ind = {motions[i]: i for i in range(len(motions))}
  vote_matrix = np.zeros((num_reps, len(motions)))

  df_grouped = vote_data.groupby(['idVotacao', 'deputado_nome'])
  for group, df_group in df_grouped:
      voto = df_group['voto'].values[0]
      i = rep_to_ind[group[1]]
      j = motion_to_ind[group[0]]
      if voto == "Sim":
          vote_matrix[i,j] = 1
      elif voto == "Não":
          vote_matrix[i,j] = -1
  return vote_matrix

def get_edges_from_vote_matrix(vote_matrix):
  M = generalized_similarity(vote_matrix)
  edges = []
  for dep1, dep2 in combinations(range(len(reps)), 2):
      if M[dep1,dep2] > 0:
          edges.append(((dep1,dep2), M[dep1,dep2]))
  return edges

edges_per_date_interval = []

for (start_date, end_date) in date_intervals:
  start_date = pd.to_datetime(start_date)
  end_date = pd.to_datetime(end_date)
  vote_matrix = get_vote_matrix(votes, start_date, end_date)
  edges = get_edges_from_vote_matrix(vote_matrix)
  edges_per_date_interval.append(edges)
  print(len(edges))

122806
102877
97045
97446
102926
106299
114736
88385


In [ ]:
graphs = []
for i, (start_date, end_date) in enumerate(date_intervals):
  print(f"Graph for {start_date} to {end_date}.")

  # --- Original igraph graph ---
  graph = Graph(graph_attrs={'name': 'Câmara dos Deputados'}, directed=False)
  graph.add_vertices(reps)  # reps is your list of names

  edges = edges_per_date_interval[i]

  filer_edges, weights = filter_edges(edges, num_nodes=graph.vcount(), threshold=None, density=1)
  graph.add_edges(filer_edges)
  graph.es['weight'] = weights
  graph.es['similarity'] = graph.es['weight']
  # Normalize weights to [0,1]
  maxw = max(graph.es['weight'])
  minw = min(graph.es['weight'])
  graph.es['weight'] = [(e - minw) / (maxw - minw) for e in graph.es['weight']]
  graph.es['date'] = [pd.to_datetime(start_date) for _ in range(graph.ecount())]


  print(graph.vcount(), graph.ecount())

  # --- Convert to NetworkX while preserving names ---
  G_nx = graph.to_networkx()  # igraph uses indices as nodes here
  mapping = {v.index: v["name"] for v in graph.vs}  # index → name
  G_nx = nx.relabel_nodes(G_nx, mapping)

  # --- Apply netbone filters ---
  backbone = disparity(G_nx, weight='weight')
  # G_nx = threshold_filter(backbone, 0.4)
  G_nx = fraction_filter(backbone, 0.3)

  # --- Convert back to igraph, restoring names ---
  graph = Graph.from_networkx(G_nx)
  graph.vs["name"] = list(G_nx.nodes)  # explicitly restore names

  graphs.append(graph)

  print(graph.vcount(), graph.ecount())

Graph for 2018-01-01 to 2018-12-31.
1166 122806
Disparity Filter
553 36842
Graph for 2019-01-01 to 2019-12-31.
1166 102877
Disparity Filter
533 30864
Graph for 2020-01-01 to 2020-12-31.
1166 97045
Disparity Filter
546 29114
Graph for 2021-01-01 to 2021-12-31.
1166 97446
Disparity Filter
532 29234
Graph for 2022-01-01 to 2022-12-31.
1166 102926
Disparity Filter
558 30878
Graph for 2023-01-01 to 2023-12-31.
1166 106299
Disparity Filter
551 31890
Graph for 2024-01-01 to 2024-12-31.
1166 114736
Disparity Filter
559 34421
Graph for 2025-01-01 to 2025-12-31.
1166 88385
Disparity Filter
531 26516


In [ ]:
# Step 1: union of node sets
all_nodes = set()
for g in graphs:
  all_nodes |= set(g.vs["name"])
all_nodes = sorted(all_nodes)
graph = Graph(directed=False)
graph.add_vertices(all_nodes)

# Helper to add edges with attributes
def add_edges_with_attrs(target_graph, source_graph):
    for e in source_graph.es:
        s_name = source_graph.vs[e.source]["name"]
        t_name = source_graph.vs[e.target]["name"]
        s_idx = target_graph.vs.find(name=s_name).index
        t_idx = target_graph.vs.find(name=t_name).index
        # Add edge and get its index
        new_eid = target_graph.add_edge(s_idx, t_idx).index
        # Copy each attribute
        for attr in source_graph.es.attributes():
            target_graph.es[new_eid][attr] = e[attr]

for g in graphs:
  print("Merging graph.")
  add_edges_with_attrs(graph, g)

In [ ]:
graph.vcount(), graph.ecount()

(1165, 249759)

In [ ]:
# Community Detection
graph.vs['community'] = graph.community_leiden(weights='weight', objective_function='modularity', resolution_parameter=1, n_iterations=100).membership
graph.vs['community_leiden'] = [str(c) for c in graph.vs['community']]
print(graph.vs['community'])
print(graph.modularity(graph.vs['community'], weights='weight'))

[0, 1, 2, 2, 3, 0, 1, 0, 1, 4, 1, 1, 2, 1, 0, 2, 0, 3, 4, 2, 4, 3, 4, 3, 1, 2, 0, 0, 0, 3, 4, 4, 1, 3, 3, 2, 4, 0, 3, 4, 4, 3, 1, 2, 4, 2, 1, 1, 4, 4, 3, 2, 0, 2, 0, 2, 0, 0, 0, 2, 0, 4, 4, 4, 1, 0, 4, 3, 1, 2, 2, 4, 3, 4, 3, 3, 4, 0, 1, 3, 1, 1, 2, 1, 0, 0, 1, 2, 1, 1, 4, 4, 3, 1, 1, 3, 3, 0, 1, 4, 1, 1, 3, 0, 1, 3, 0, 4, 3, 4, 0, 4, 0, 1, 1, 1, 4, 1, 0, 0, 0, 3, 1, 3, 2, 2, 3, 4, 3, 4, 1, 3, 3, 3, 3, 1, 1, 0, 0, 4, 2, 1, 1, 3, 4, 1, 0, 1, 4, 4, 2, 2, 2, 2, 3, 0, 3, 4, 3, 2, 1, 3, 0, 1, 3, 0, 2, 1, 1, 0, 1, 4, 4, 3, 4, 2, 0, 3, 1, 3, 4, 0, 3, 1, 0, 3, 2, 4, 4, 4, 1, 4, 0, 2, 3, 3, 3, 2, 0, 3, 4, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 3, 4, 4, 4, 3, 4, 1, 1, 1, 0, 3, 4, 4, 0, 4, 4, 0, 2, 4, 4, 3, 2, 2, 2, 2, 1, 2, 3, 4, 3, 2, 0, 3, 4, 2, 4, 2, 0, 1, 4, 0, 0, 3, 0, 2, 1, 2, 1, 0, 0, 2, 2, 2, 2, 2, 0, 2, 1, 2, 4, 4, 1, 2, 2, 0, 0, 0, 0, 4, 0, 4, 3, 2, 4, 0, 0, 0, 2, 3, 0, 0, 2, 4, 2, 3, 0, 2, 1, 2, 3, 0, 0, 3, 0, 3, 0, 2, 2, 3, 0, 4, 0, 4, 3, 1, 4, 0, 3, 3, 1, 4, 3, 3, 4, 2, 3, 3, 3, 

In [ ]:
# Add info to nodes
info = [votes[votes['deputado_nome']==dep] for dep in graph.vs['name']]
graph.vs['Foto'] = [x['deputado_urlFoto'].values[-1] for x in info]
graph.vs['UF'] = [x['deputado_siglaUf'].values[-1] for x in info]
graph.vs['Partido'] = [x['deputado_siglaPartido'].values[-1] for x in info]
graph.vs['URL'] =  [x['deputado_uri'].values[-1] for x in info]
graph.vs['Partido URI'] =  [x['deputado_uriPartido'].values[-1] for x in info]
# graph.vs['Total Despesas (reais)'] = [get_total_expenses(x['deputado_id'].values[-1], start_date, end_date) for x in info]
#for i, x in enumerate(info):
#    for k, v in get_deputy_info(x['deputado_id'].values[-1]).items():
#        if i == 0:
#            graph.vs[k] = ['' for _ in range(len(graph.vs))]
#        graph.vs[i][k] = v

graph.vs['betweenness'] = graph.betweenness()
graph.vs['closeness'] = graph.closeness()

summary(graph)

IGRAPH UNW- 1165 249759 -- 
+ attr: Foto (v), Partido (v), Partido URI (v), UF (v), URL (v), betweenness (v), closeness (v), community (v), community_leiden (v), name (v), date (e), p_value (e), similarity (e), weight (e)


## Visualization

In [ ]:
g = graphistry.bind(source='src', destination='dst',
                    point_label='name',
                    edge_weight='similarity',
                    edge_label='similarity',
                    edge_size='similarity',
                    point_color='community',
                    point_size='name')

(e_df, n_df) = g.igraph2pandas(graph)

g = g.nodes(n_df).edges(e_df)
g = g.nodes(lambda g: g._nodes.assign(community=g._nodes['community'].astype('int32')))
g = g.addStyle(bg={'color': 'white'}, page={'title': 'Câmara dos Deputados'})
g = g.encode_point_icon('Partido URI', categorical_mapping=party_logo, shape='circle')
g = g.settings(url_params={
    'pageTitle': 'Câmara dos Deputados',
    'play': 2000,
    'menu': True,
    'info': True,
    'strongGravity': True,
    'showArrows': False,
    'pointSize': 5.0,
    'pointsOfInterestMax': 100,
    'edgeInfluence': 1.0,
    'showLabels': True, 'showLabelOnHover': True,
    'showPointsOfInterest': True, 'showPointsOfInterestLabel': True, 'showLabelPropertiesOnHover': True})
g.plot(render=True)

DEBUG:graphistry.PlotterBase:1. @PloatterBase plot: _pygraphistry.org_name: lucasalegre
DEBUG:graphistry.PlotterBase:2. @PloatterBase plot: self._pygraphistry.org_name: lucasalegre
DEBUG:graphistry.PlotterBase:3. @PloatterBase plot: self._pygraphistry.org_name: lucasalegre
DEBUG:graphistry.pygraphistry:1. @PyGraphistry refresh, org_name: lucasalegre
DEBUG:graphistry.pygraphistry:JWT refresh via token
DEBUG:graphistry.arrow_uploader:2. @ArrowUploader.__init__: After set self.org_name: lucasalegre, self.__org_name : lucasalegre
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): hub.graphistry.com:443
DEBUG:urllib3.connectionpool:https://hub.graphistry.com:443 "POST /api/v2/auth/token/refresh HTTP/1.1" 200 1197
DEBUG:graphistry.PlotterBase:4. @PloatterBase plot: self._pygraphistry.org_name: lucasalegre
DEBUG:graphistry.PlotterBase:_make_dataset (mode arrow, memoize True) name:[Untitled 995HBB3SHS] des:[] (e::<class 'pandas.core.frame.DataFrame'>, n::<class 'pandas.core.frame.